### 임포트

In [1]:
import torch
import os
import sys
import numpy as np
import random
import copy

# custom
from util import *
from transformers import GPT2Tokenizer
from AAC_Prefix.AAC_Prefix import * # network
from Train import *

### 기타 값들 설정

In [2]:
# prefix vector 크기 설정
temporal_prefix_size = 15 # 0 or 15
global_prefix_size = 11 # 0 or 11

prefix_size = temporal_prefix_size + global_prefix_size 

# mapping network가 사용할 transformer의 스펙 설정
transformer_num_layers = {"temporal_num_layers" : 4, "global_num_layers" : 4}
prefix_size_dict = {"temporal_prefix_size" : temporal_prefix_size, "global_prefix_size" : global_prefix_size}


data_dir = './Clotho'
MODEL_NAME = 'add_exp_train_clotho_test_clotho_using_pretrained_mappingnetwork_in_addexp1'

epochs = 60
LR = 5e-5

TEST_BATCH_SIZE = 5
TRAIN_BATCH_SIZE = 55

random_seed=2766
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.benchmark=False
torch.backends.cudnn.deterministic=True
np.random.seed(random_seed)
random.seed(random_seed)  

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda' if USE_CUDA else 'cpu')

### Tokenizer, Dataloader 불러오기

In [3]:
tokenizer_type = 'Custom'
tokenizer = tokenizer_forCustomVocab(Dataset = 'Clotho')
vocab_size = len(tokenizer.vocab)

In [4]:
test_dataloader  = CreateDataloader(tokenizer, data_dir, TEST_BATCH_SIZE, 'evaluation', prefix_size, is_TrainDataset = False, tokenizer_type = tokenizer_type)
train_dataloader = CreateDataloader(tokenizer, data_dir, TRAIN_BATCH_SIZE, 'development', prefix_size, is_TrainDataset = True, tokenizer_type = tokenizer_type)

get dataset...: 100%|██████████████████████| 2893/2893 [00:22<00:00, 128.14it/s]


### 학습결과 정리하는 폴더 생성하기

In [5]:
directory = "./Train_record/params_" + MODEL_NAME
try:
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print("Error: Failed to create the directory.")


### 모델 초기화

In [6]:
model = get_AAC_Prefix(tokenizer, 
                        vocab_size = vocab_size, Dataset = 'Clotho',
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = True, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

# pre_trained된 mapping network가져와야함
tokenizer_addexp1 = tokenizer_forCustomVocab(Dataset = 'AudioCaps')
vocab_size_addexp1 = len(tokenizer_addexp1.vocab)


model_in_addexp1 = get_AAC_Prefix(tokenizer_addexp1, 
                        vocab_size = vocab_size_addexp1, Dataset = 'AudioCaps',
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = True, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

params_path = './Train_record/params_add_exp_train_audiocaps_test_audiocaps/Param_epoch_39.pt'
params = torch.load(params_path, map_location = device)
model_in_addexp1.load_state_dict(params) 

model.temporal_mappingnetwork = copy.deepcopy(model_in_addexp1.temporal_mappingnetwork)
model.global_mappingnetwork = copy.deepcopy(model_in_addexp1.global_mappingnetwork)

del model_in_addexp1, params_path, params


/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


use Custom Tokenizer
temporal feature's mapping network : num_head = 8 num_layers = 4
global feature ver's mapping network : num_head = 8 num_layers = 4
use custom header!
Encoder freezing
GPT2 freezing
use Custom Tokenizer
temporal feature's mapping network : num_head = 8 num_layers = 4
global feature ver's mapping network : num_head = 8 num_layers = 4
use custom header!
Encoder freezing
GPT2 freezing


### 학습 & 평가

In [7]:
Train(model, LR, train_dataloader, test_dataloader,
    epochs, model_name = MODEL_NAME, beam_search = True, device = device,
    Dataset = 'Clotho', test_dataloader_other_dataset = None)

/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Eval using dataset...: 100%|████████████████| 1045/1045 [03:56<00:00,  4.42it/s]


loading annotations into memory...
0:00:00.007305
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 70696 tokens at 475937.67 tokens per second.
PTBTokenizer tokenized 12148 tokens at 135035.94 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 10059, 'reflen': 10491, 'guess': [10059, 9014, 7969, 6924], 'correct': [5343, 1966, 666, 139]}
ratio: 0.9588218472975923
Bleu_1: 0.509
Bleu_2: 0.326
Bleu_3: 0.204
Bleu_4: 0.113
computing METEOR score...
METEOR: 0.146
computing Rouge score...
ROUGE_L: 0.351
computing CIDEr score...
CIDEr: 0.269
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.4 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.6

...

loading annotations into memory...
0:00:00.006816
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 70696 tokens at 466463.27 tokens per second.
PTBTokenizer tokenized 11905 tokens at 134815.06 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 9813, 'reflen': 10254, 'guess': [9813, 8768, 7723, 6678], 'correct': [5551, 2173, 793, 214]}
ratio: 0.9569923932123116
Bleu_1: 0.541
Bleu_2: 0.358
Bleu_3: 0.233
Bleu_4: 0.140
computing METEOR score...
METEOR: 0.156
computing Rouge score...
ROUGE_L: 0.366
computing CIDEr score...
CIDEr: 0.316
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.4 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.6

SPICE evaluation took: 10.71 s
SPICE: 0.106
computing SPIDEr score...
SPIDEr: 0.211

Training time : 2:12:19
